In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount = True)

Mounted at /content/gdrive


In [ ]:
!pip install pandasql
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as sm
import numpy as np
import matplotlib.pyplot as plt
from sqlite3 import connect
from scipy.stats.mstats import winsorize, ks_2samp
import seaborn as sns
import pandasql as ps

conn = connect(':memory:', timeout=10)
filePath = "/content/gdrive/MyDrive/FinancePaperFIMA"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
file = filePath + "/CRSPMonthly1990Through2022.csv"
crsp1990to2022 = pd.read_csv(file)
print(crsp1990to2022)
print(crsp1990to2022.columns)


<ipython-input-3-7bb977c5e1bc>:2: DtypeWarning: Columns (5,9,18,19,49,58) have mixed types. Specify dtype option on import or set low_memory=False.
  crsp1990to2022 = pd.read_csv(file)


         PERMNO      date  NAMEENDT  SHRCD  EXCHCD   SICCD    NCUSIP TICKER  \
0         10001  19900131       NaN   11.0     3.0    4920  39040610   GFGC   
1         10001  19900228       NaN   11.0     3.0    4920  39040610   GFGC   
2         10001  19900330       NaN   11.0     3.0    4920  39040610   GFGC   
3         10001  19900430       NaN   11.0     3.0    4920  39040610   GFGC   
4         10001  19900531       NaN   11.0     3.0    4920  39040610   GFGC   
...         ...       ...       ...    ...     ...     ...       ...    ...   
2975236   93436  20211130       NaN   11.0     3.0  9999.0  88160R10   TSLA   
2975237   93436  20211231       NaN   11.0     3.0  9999.0  88160R10   TSLA   
2975238   93436  20220131       NaN   11.0     3.0  9999.0  88160R10   TSLA   
2975239   93436  20220228       NaN   11.0     3.0  9999.0  88160R10   TSLA   
2975240   93436  20220331       NaN   11.0     3.0  9999.0  88160R10   TSLA   

                     COMNAM SHRCLS  ... CFACSHR    

In [ ]:
crsp1990to2022.to_sql("crsp1990to2022", conn, if_exists='replace')
query = '''
        SELECT DISTINCT PERMNO as permno, date, RET as ret, SHROUT as shrout, ALTPRC as altprc, EXCHCD as exchcd,
        SHRCD as shrcd, SICCD as siccd, DLRET as dlret, DLSTCD as dlstcd
        FROM crsp1990to2022
        '''
crsp1990to2022 = pd.read_sql(query, conn)

crsp1990to2022['permno']=pd.to_numeric(crsp1990to2022['permno'], errors='coerce')
crsp1990to2022['date']=pd.to_datetime(crsp1990to2022['date'], format='%Y%m%d')
crsp1990to2022['ret']=pd.to_numeric(crsp1990to2022['ret'], errors='coerce')
crsp1990to2022['shrout']=pd.to_numeric(crsp1990to2022['shrout'], errors='coerce')
crsp1990to2022['altprc']=pd.to_numeric(crsp1990to2022['altprc'], errors='coerce')
crsp1990to2022['exchcd']=pd.to_numeric(crsp1990to2022['exchcd'], errors='coerce')
crsp1990to2022['shrcd']=pd.to_numeric(crsp1990to2022['shrcd'], errors='coerce')
crsp1990to2022['siccd']=pd.to_numeric(crsp1990to2022['siccd'], errors='coerce')
crsp1990to2022['dlret']=pd.to_numeric(crsp1990to2022['dlret'], errors='coerce')
crsp1990to2022['dlstcd']=pd.to_numeric(crsp1990to2022['dlstcd'], errors='coerce')

crsp1990to2022['ret'] *= 100
crsp1990to2022['dlret'] *= 100


In [ ]:
crsp2018to2022 = crsp1990to2022[crsp1990to2022['date'] >= "2018-01-01 00:00:00"]

crsp2018to2022 = crsp2018to2022[(crsp2018to2022['shrcd']==10) | (crsp2018to2022['shrcd']==11)]

In [ ]:
crsp2018to2022['market_cap'] = abs(crsp2018to2022['shrout'] * crsp2018to2022['altprc'] / 1000)
crsp2018to2022['market_cap'].replace(0, np.NaN)

2358       2.585782e+03
2359       2.509016e+03
2360       2.553262e+03
2361       2.569842e+03
2362       2.648577e+03
               ...     
2947235    1.149642e+06
2947236    1.092218e+06
2947237    9.681319e+05
2947238    8.996190e+05
2947239    1.113736e+06
Name: market_cap, Length: 194624, dtype: float64

In [ ]:
crsp2018to2022['exchange'] = ""
for i in crsp2018to2022.index:
  if crsp2018to2022['exchcd'][i] in [1,31]:
    crsp2018to2022['exchange'][i] = "NYSE"
  elif crsp2018to2022['exchcd'][i] in [2,32]:
    crsp2018to2022['exchange'][i] = "AMEX"
  elif crsp2018to2022['exchcd'][i] in [3,33]:
    crsp2018to2022['exchange'][i] = "NASDAQ"
  else:
    crsp2018to2022['exchange'][i] = "Other"
crsp2018to2022 = crsp2018to2022[crsp2018to2022['exchange'] != "Other"]


<ipython-input-7-05d956f2c751>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp2018to2022['exchange'][i] = "NASDAQ"
<ipython-input-7-05d956f2c751>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp2018to2022['exchange'][i] = "AMEX"
<ipython-input-7-05d956f2c751>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp2018to2022['exchange'][i] = "Other"
<ipython-input-7-05d956f2c751>:4: SettingWithCopyWarning: 
A value is trying t

In [ ]:
crsp2018to2022['ret_adj'] = ""
for i in crsp2018to2022.index:
  if pd.isnull(crsp2018to2022['dlstcd'][i]):
    crsp2018to2022['ret_adj'][i] = crsp2018to2022['ret'][i]
  elif(pd.notnull(crsp2018to2022['dlstcd'][i])) and (pd.notnull(crsp2018to2022['dlret'][i])):
    crsp2018to2022['ret_adj'][i] = crsp2018to2022['dlret'][i]
  elif(551 <= crsp2018to2022['dlret'][i] <= 574) or (crsp2018to2022['dlret'][i] in [500, 520, 580, 584]):
    crsp2018to2022['ret_adj'][i] = -30
  else:
    crsp2018to2022['ret_adj'][i] = -100

<ipython-input-8-1a63a5816ab7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp2018to2022['ret_adj'] = ""
<ipython-input-8-1a63a5816ab7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp2018to2022['ret_adj'][i] = crsp2018to2022['ret'][i]
<ipython-input-8-1a63a5816ab7>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp2018to2022['ret_adj'][i] = -100
<ipython-input-8-1

In [ ]:
crsp2018to2022 = crsp2018to2022.reset_index()
crsp2018to2022.drop(['shrcd', 'dlret', 'dlstcd', 'index'], inplace = True, axis = 1)

print(crsp2018to2022)
print(crsp2018to2022.columns)

        permno       date      ret     shrout      altprc  exchcd   siccd  \
0        10026 2018-01-31  -8.8191    18678.0   138.44000     3.0  2052.0   
1        10026 2018-02-28  -2.9688    18678.0   134.33000     3.0  2052.0   
2        10026 2018-03-29   1.9951    18697.0   136.56000     3.0  2052.0   
3        10026 2018-04-30   0.6224    18702.0   137.41000     3.0  2052.0   
4        10026 2018-05-31   3.0638    18702.0   141.62000     3.0  2052.0   
...        ...        ...      ...        ...         ...     ...     ...   
193370   93436 2021-11-30   2.7612  1004265.0  1144.76001     3.0  9999.0   
193371   93436 2021-12-31  -7.6855  1033534.0  1056.78003     3.0  9999.0   
193372   93436 2022-01-31 -11.3609  1033534.0   936.71997     3.0  9999.0   
193373   93436 2022-02-28  -7.0768  1033534.0   870.42999     3.0  9999.0   
193374   93436 2022-03-31  23.8009  1033534.0  1077.59998     3.0  9999.0   

          market_cap exchange  ret_adj  
0       2.585782e+03   NASDAQ  -8.

In [ ]:
file = filePath + "/ccmfund1990to2022.csv"
ccmfund1990to2022 = pd.read_csv(file)
print(ccmfund1990to2022)
print(ccmfund1990to2022.columns)

<ipython-input-10-62969edd8ac4>:2: DtypeWarning: Columns (2,17,19,24,33,37,952,953,955,956,957,962,976,977,982) have mixed types. Specify dtype option on import or set low_memory=False.
  ccmfund1990to2022 = pd.read_csv(file)


         GVKEY LINKPRIM LIID LINKTYPE  LPERMNO  LPERMCO    LINKDT LINKENDDT  \
0         1004        P    1       LU    54594    20000  19720424         E   
1         1004        P    1       LU    54594    20000  19720424         E   
2         1004        P    1       LU    54594    20000  19720424         E   
3         1004        P    1       LU    54594    20000  19720424         E   
4         1004        P    1       LU    54594    20000  19720424         E   
...        ...      ...  ...      ...      ...      ...       ...       ...   
209121  345980        P    1       LC    20333    57666  20201216         E   
209122  345980        P    1       LC    20333    57666  20201216         E   
209123  347007        P    1       LC    15533    55364  20210310         E   
209124  347085        P    1       LC    21069    58208  20210401         E   
209125  349530        P    1       LC    17324    56274  20210709         E   

        datadate   fyear  ... priusa     sic spcind

In [ ]:
ccmfund1990to2022.to_sql("ccmfund1990to2022", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT GVKEY, LPERMNO, datadate, LINKTYPE, LINKENDDT, seq, ceq, at, lt, txditc,
        txdb, itcb, pstkrv, pstkl, pstk, indfmt, datafmt
        FROM ccmfund1990to2022
        '''
compustat1990to2022 = pd.read_sql(query, conn)

In [ ]:
compustat1990to2022['GVKEY'] = pd.to_numeric(compustat1990to2022['GVKEY'], errors='coerce')
compustat1990to2022['LPERMNO'] = pd.to_numeric(compustat1990to2022['LPERMNO'], errors='coerce')
compustat1990to2022['datadate'] = pd.to_datetime(compustat1990to2022['datadate'], format = '%Y%m%d')
compustat1990to2022['LINKTYPE'] = compustat1990to2022['LINKTYPE'].apply(str)
compustat1990to2022['LINKENDDT'] = pd.to_datetime(compustat1990to2022['LINKENDDT'], format = "%Y%m%d", errors = 'coerce')
compustat1990to2022['seq'] = pd.to_numeric(compustat1990to2022['seq'], errors='coerce')
compustat1990to2022['ceq'] = pd.to_numeric(compustat1990to2022['ceq'], errors='coerce')
compustat1990to2022['at'] = pd.to_numeric(compustat1990to2022['at'], errors='coerce')
compustat1990to2022['lt'] = pd.to_numeric(compustat1990to2022['lt'], errors='coerce')
compustat1990to2022['txditc'] = pd.to_numeric(compustat1990to2022['txditc'], errors='coerce')
compustat1990to2022['txdb'] = pd.to_numeric(compustat1990to2022['txdb'], errors='coerce')
compustat1990to2022['itcb'] = pd.to_numeric(compustat1990to2022['itcb'], errors='coerce')
compustat1990to2022['pstkrv'] = pd.to_numeric(compustat1990to2022['pstkrv'], errors='coerce')
compustat1990to2022['pstkl'] = pd.to_numeric(compustat1990to2022['pstkl'], errors='coerce')
compustat1990to2022['pstk'] = pd.to_numeric(compustat1990to2022['pstk'], errors='coerce')
compustat1990to2022['indfmt'] = compustat1990to2022['indfmt'].apply(str)
compustat1990to2022['datafmt'] = compustat1990to2022['datafmt'].apply(str)


In [ ]:
compustat2018to2022 = compustat1990to2022[compustat1990to2022['datadate'] >= "2018-01-01 00:00:00"]

compustat2018to2022 = compustat2018to2022[compustat2018to2022['indfmt'] == "INDL"]
compustat2018to2022 = compustat2018to2022[compustat2018to2022['datafmt'] == "STD"]

compustat2018to2022 = compustat2018to2022[(compustat2018to2022['LINKTYPE']== "LU") | (compustat2018to2022['LINKTYPE'] == "LC")]

compustat2018to2022 = compustat2018to2022[(compustat2018to2022['datadate'] <= compustat2018to2022['LINKENDDT']) | pd.isnull(compustat2018to2022['LINKENDDT'])]

In [ ]:
compustat2018to2022.to_sql("compustat2018to2022", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT GVKEY as gvkey, LPERMNO as permno, datadate,
        COALESCE(seq, ceq + pstk, at - lt) + COALESCE(txditc, txdb + itcb, 0) - COALESCE(pstkrv, pstkl, pstk, 0) as book_value
        FROM compustat2018to2022
        '''
compustat2018to2022 = pd.read_sql(query, conn)
compustat2018to2022['book_value'] = pd.to_numeric(compustat2018to2022['book_value'], errors = 'coerce')


In [ ]:
compustat2018to2022['datadate'] = pd.to_datetime(compustat2018to2022['datadate'])
compustat2018to2022['year'] = compustat2018to2022['datadate'].dt.year
compustat2018to2022 = compustat2018to2022.sort_values(by = ['permno', 'datadate'], ascending = True)
compustat2018to2022 = compustat2018to2022.reset_index()
toDrop = []
for i in range(len(compustat2018to2022) - 1):
  if(compustat2018to2022['permno'][i] == compustat2018to2022['permno'][i+1]) and (compustat2018to2022['year'][i] == compustat2018to2022['year'][i+1]):
    toDrop.append(i)
compustat2018to2022.drop(toDrop, inplace = True, axis = 0)
compustat2018to2022.drop('index', inplace = True, axis = 1)

In [ ]:
compustat2018to2022.replace([np.inf, -np.inf], np.nan, inplace = True)
compustat2018to2022 = compustat2018to2022.dropna()

compustat2018to2022 = compustat2018to2022.reset_index()
compustat2018to2022.drop('index', inplace = True, axis = 1)

In [ ]:
compustat2018to2022['year'] = pd.to_numeric(compustat2018to2022['year'], errors = 'coerce')
compustat2018to2022['reference_date'] = compustat2018to2022['year'] + 1
compustat2018to2022['reference_date'] = compustat2018to2022['reference_date'].apply(str)
compustat2018to2022['reference_date'] += "-06-01"
compustat2018to2022['reference_date'] = pd.to_datetime(compustat2018to2022['reference_date'])
compustat2018to2022.drop('year', inplace = True, axis = 1)

print(compustat2018to2022)
print(compustat2018to2022.columns)

        gvkey  permno   datadate  book_value reference_date
0       12825   10026 2018-09-30     811.413     2019-06-01
1       12825   10026 2019-09-30     895.671     2020-06-01
2       12825   10026 2020-09-30     873.911     2021-06-01
3       12825   10026 2021-09-30     907.232     2022-06-01
4       12096   10028 2018-12-31       8.413     2019-06-01
...       ...     ...        ...         ...            ...
20524  184259   93434 2021-06-30      74.602     2022-06-01
20525  184996   93436 2018-12-31    4923.243     2019-06-01
20526  184996   93436 2019-12-31    6618.000     2020-06-01
20527  184996   93436 2020-12-31   22376.000     2021-06-01
20528  184996   93436 2021-12-31   30213.000     2022-06-01

[20529 rows x 5 columns]
Index(['gvkey', 'permno', 'datadate', 'book_value', 'reference_date'], dtype='object')


In [ ]:
crsp2018to2022.to_sql("crsp2018to2022", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT permno, date, exchange, ret_adj as ret, market_cap
        FROM crsp2018to2022
        '''
stocks2018to2022 = pd.read_sql(query, conn)

stocks2018to2022.replace([np.inf, -np.inf], np.nan, inplace = True)
stocks2018to2022 = stocks2018to2022.dropna()

In [ ]:
stocks2018to2022['date'] = pd.to_datetime(stocks2018to2022['date'])
stocks2018to2022['reference_date'] = ""
for i in stocks2018to2022.index:
  if stocks2018to2022['date'][i].month < 6:
    stocks2018to2022['reference_date'][i] = str(stocks2018to2022['date'][i].year - 1) + "-06-01"
  else:
    stocks2018to2022['reference_date'][i] = str(stocks2018to2022['date'][i].year) + "-06-01"
stocks2018to2022['reference_date'] = pd.to_datetime(stocks2018to2022['reference_date'])

<ipython-input-19-e644673460ad>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocks2018to2022['date'] = pd.to_datetime(stocks2018to2022['date'])
<ipython-input-19-e644673460ad>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocks2018to2022['reference_date'] = ""
<ipython-input-19-e644673460ad>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [ ]:
stocks2018to2022.to_sql("stocks2018to2022", conn, if_exists = 'replace')
compustat2018to2022.to_sql("compustat2018to2022", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT stocks2018to2022.*, compustat2018to2022.gvkey, compustat2018to2022.datadate,
        compustat2018to2022.book_value
        FROM stocks2018to2022
        LEFT JOIN compustat2018to2022
        ON stocks2018to2022.permno = compustat2018to2022.permno
        AND stocks2018to2022.reference_date = compustat2018to2022.reference_date
        '''
stocks2018to2022 = pd.read_sql(query, conn)
stocks2018to2022.drop('index', inplace = True, axis = 1)

In [ ]:
stocks2018to2022['date'] = pd.to_datetime(stocks2018to2022['date'])
stocksmarketequity = stocks2018to2022[stocks2018to2022['date'].dt.month == 12]
stocksmarketequity['reference_date'] = stocksmarketequity['date'].dt.year + 1
stocksmarketequity['reference_date'] = stocksmarketequity['reference_date'].apply(str)
stocksmarketequity['reference_date'] += "-06-01"
stocksmarketequity['reference_date'] = pd.to_datetime(stocksmarketequity['reference_date'])
stocksmarketequity.to_sql("stocksmarketequity", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT permno, reference_date, market_cap as market_equity
        FROM stocksmarketequity
        '''
stocksmarketequity = pd.read_sql(query, conn)
stocks2018to2022.to_sql("stocks2018to2022", conn, if_exists = 'replace')
stocksmarketequity.to_sql("stocksmarketequity", conn, if_exists = 'replace')
query = '''SELECT DISTINCT stocks2018to2022.*, stocksmarketequity.market_equity
        FROM stocks2018to2022
        LEFT JOIN stocksmarketequity
        ON stocks2018to2022.permno = stocksmarketequity.permno
        AND stocks2018to2022.reference_date = stocksmarketequity.reference_date
        '''
stocks2018to2022 = pd.read_sql(query, conn)
stocks2018to2022.drop('index', inplace = True, axis = 1)
stocks2018to2022 = stocks2018to2022.dropna()
stocks2018to2022 = stocks2018to2022.reset_index()
stocks2018to2022.drop('index', inplace = True, axis = 1)

<ipython-input-21-fd7977da6e23>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocksmarketequity['reference_date'] = stocksmarketequity['date'].dt.year + 1
<ipython-input-21-fd7977da6e23>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocksmarketequity['reference_date'] = stocksmarketequity['reference_date'].apply(str)
<ipython-input-21-fd7977da6e23>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

In [ ]:
stocks2018to2022['market_equity'] = pd.to_numeric(stocks2018to2022['market_equity'], errors = 'coerce')
stocks2018to2022['bm_ratio'] = stocks2018to2022['book_value']/stocks2018to2022['market_equity']

stocks2018to2022['date'] = pd.to_datetime(stocks2018to2022['date'])
stocksweight = stocks2018to2022[stocks2018to2022['date'].dt.month == 6]
stocks2018to2022.to_sql("stocks2018to2022", conn, if_exists = 'replace')
stocksweight.to_sql("stocksweight", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT stocks2018to2022.*, stocksweight.market_cap as market_cap_weight
        FROM stocks2018to2022
        LEFT JOIN stocksweight
        ON stocks2018to2022.permno = stocksweight.permno
        AND stocks2018to2022.reference_date = stocksweight.reference_date
        '''
stocks2018to2022 = pd.read_sql(query, conn)
stocks2018to2022.drop('index', inplace = True, axis = 1)

In [ ]:
stocks2018to2022.replace([np.inf, -np.inf], np.nan, inplace = True)
stocks2018to2022 = stocks2018to2022.dropna()
stocks2018to2022['date'] = pd.to_datetime(stocks2018to2022['date'])
stocks2018to2022 = stocks2018to2022[stocks2018to2022['date'] >= '2018-01-01 00:00:00']

<ipython-input-23-e74d5a72aace>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocks2018to2022['date'] = pd.to_datetime(stocks2018to2022['date'])


In [ ]:
print(stocks2018to2022)

        permno       date exchange       ret    market_cap  \
0        10026 2019-06-28   NASDAQ     0.373  3.030689e+03   
1        10026 2019-07-31   NASDAQ   15.4644  3.501411e+03   
2        10026 2019-08-30   NASDAQ    3.8851  3.637443e+03   
3        10026 2019-09-30   NASDAQ   -0.2901  3.627840e+03   
4        10026 2019-10-31   NASDAQ   -0.6458  3.604410e+03   
...        ...        ...      ...       ...           ...   
115662   93436 2021-11-30   NASDAQ    2.7612  1.149642e+06   
115663   93436 2021-12-31   NASDAQ   -7.6855  1.092218e+06   
115664   93436 2022-01-31   NASDAQ  -11.3609  9.681319e+05   
115665   93436 2022-02-28   NASDAQ   -7.0768  8.996190e+05   
115666   93436 2022-03-31   NASDAQ      -100  1.113736e+06   

             reference_date     gvkey             datadate  book_value  \
0       2019-06-01 00:00:00   12825.0  2018-09-30 00:00:00     811.413   
1       2019-06-01 00:00:00   12825.0  2018-09-30 00:00:00     811.413   
2       2019-06-01 00:00:00   128

In [ ]:
stocks2018to2022['date'] = pd.to_datetime(stocks2018to2022['date'])
sizebreakpoints = stocks2018to2022[(stocks2018to2022['date'].dt.month == 6) & (stocks2018to2022['exchange'] == "NYSE")]
sizebreakpoints.to_sql("sizebreakpoints", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT permno, reference_date, market_cap
        FROM sizebreakpoints
        '''
sizebreakpoints = pd.read_sql(query, conn)
sizebreakpoints['reference_date'] = pd.to_datetime(sizebreakpoints['reference_date'])
g = sizebreakpoints.groupby(['reference_date'])

In [ ]:
sizesorts = stocks2018to2022[(stocks2018to2022['date'].dt.month == 6)]

sizesorts['size_median'] = ""
for i in sizesorts.index:
  sizesorts['size_median'][i] = g.get_group(g.get_group(sizesorts['reference_date'][i]).reset_index()['reference_date'][0])['market_cap'].quantile(0.5)

Streaming output truncated to the last 5000 lines.
<ipython-input-26-0f6b76b32cf8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sizesorts['size_median'][i] = g.get_group(g.get_group(sizesorts['reference_date'][i]).reset_index()['reference_date'][0])['market_cap'].quantile(0.5)
<ipython-input-26-0f6b76b32cf8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sizesorts['size_median'][i] = g.get_group(g.get_group(sizesorts['reference_date'][i]).reset_index()['reference_date'][0])['market_cap'].quantile(0.5)
<ipython-input-26-0f6b76b32cf8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [ ]:
sizesorts['size_portfolio'] = ""
for i in sizesorts.index:
  if sizesorts['market_cap'][i] > sizesorts['size_median'][i]:
    sizesorts['size_portfolio'][i] = "B"
  else:
    sizesorts['size_portfolio'][i] = "S"

<ipython-input-27-beb3343ddb75>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sizesorts['size_portfolio'] = ""
<ipython-input-27-beb3343ddb75>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sizesorts['size_portfolio'][i] = "B"
<ipython-input-27-beb3343ddb75>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sizesorts['size_portfolio'][i] = "S"


In [ ]:
stocks2018to2022.to_sql("stocks2018to2022", conn, if_exists = 'replace')
sizesorts.to_sql("sizesorts", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT stocks2018to2022.*, sizesorts.size_portfolio
        FROM stocks2018to2022
        LEFT JOIN sizesorts
        ON stocks2018to2022.reference_date = sizesorts.reference_date
        AND stocks2018to2022.permno = sizesorts.permno
        '''
stocks2018to2022 = pd.read_sql(query, conn)
stocks2018to2022.drop('index', inplace = True, axis = 1)

print(stocks2018to2022)
print(stocks2018to2022.columns)

        permno                 date exchange       ret    market_cap  \
0        10026  2019-06-28 00:00:00   NASDAQ     0.373  3.030689e+03   
1        10026  2019-07-31 00:00:00   NASDAQ   15.4644  3.501411e+03   
2        10026  2019-08-30 00:00:00   NASDAQ    3.8851  3.637443e+03   
3        10026  2019-09-30 00:00:00   NASDAQ   -0.2901  3.627840e+03   
4        10026  2019-10-31 00:00:00   NASDAQ   -0.6458  3.604410e+03   
...        ...                  ...      ...       ...           ...   
115641   93436  2021-11-30 00:00:00   NASDAQ    2.7612  1.149642e+06   
115642   93436  2021-12-31 00:00:00   NASDAQ   -7.6855  1.092218e+06   
115643   93436  2022-01-31 00:00:00   NASDAQ  -11.3609  9.681319e+05   
115644   93436  2022-02-28 00:00:00   NASDAQ   -7.0768  8.996190e+05   
115645   93436  2022-03-31 00:00:00   NASDAQ      -100  1.113736e+06   

             reference_date     gvkey             datadate  book_value  \
0       2019-06-01 00:00:00   12825.0  2018-09-30 00:00:00   

In [ ]:
stocks2018to2022['date'] = pd.to_datetime(stocks2018to2022['date'])
valuebreakpoints = stocks2018to2022[(stocks2018to2022['date'].dt.month == 6) & (stocks2018to2022['exchange'] == "NYSE")]
valuebreakpoints.to_sql("valuebreakpoints", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT permno, reference_date, bm_ratio
        FROM valuebreakpoints
        '''
valuebreakpoints = pd.read_sql(query, conn)
valuebreakpoints['reference_date'] = pd.to_datetime(valuebreakpoints['reference_date'])
g = valuebreakpoints.groupby(['reference_date'])

In [ ]:
valuesorts = stocks2018to2022[(stocks2018to2022['date'].dt.month == 6)]

valuesorts['value_q30'] = ""
valuesorts['value_q70'] = ""
for i in valuesorts.index:
  valuesorts['value_q30'][i] = g.get_group(g.get_group(valuesorts['reference_date'][i]).reset_index()['reference_date'][0])['bm_ratio'].quantile(0.3)
  valuesorts['value_q70'][i] = g.get_group(g.get_group(valuesorts['reference_date'][i]).reset_index()['reference_date'][0])['bm_ratio'].quantile(0.7)

Streaming output truncated to the last 5000 lines.
<ipython-input-30-a9499d7f4770>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valuesorts['value_q30'][i] = g.get_group(g.get_group(valuesorts['reference_date'][i]).reset_index()['reference_date'][0])['bm_ratio'].quantile(0.3)
<ipython-input-30-a9499d7f4770>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valuesorts['value_q70'][i] = g.get_group(g.get_group(valuesorts['reference_date'][i]).reset_index()['reference_date'][0])['bm_ratio'].quantile(0.7)
<ipython-input-30-a9499d7f4770>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

In [ ]:
valuesorts['value_portfolio'] = ""
for i in valuesorts.index:
  if valuesorts['bm_ratio'][i] > valuesorts['value_q70'][i]:
    valuesorts['value_portfolio'][i] = "H"
  elif valuesorts['value_q30'][i] < valuesorts['bm_ratio'][i] <= valuesorts['value_q70'][i]:
    valuesorts['value_portfolio'][i] = "M"
  else:
    valuesorts['value_portfolio'][i] = "L"

<ipython-input-31-b3917dde1a14>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valuesorts['value_portfolio'] = ""
<ipython-input-31-b3917dde1a14>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valuesorts['value_portfolio'][i] = "L"
<ipython-input-31-b3917dde1a14>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valuesorts['value_portfolio'][i] = "M"
<ipython-input-31-b3917dde

In [ ]:
stocks2018to2022.to_sql("stocks2018to2022", conn, if_exists = 'replace')
valuesorts.to_sql("valuesorts", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT stocks2018to2022.*, valuesorts.value_portfolio
        FROM stocks2018to2022
        LEFT JOIN valuesorts
        ON stocks2018to2022.reference_date = valuesorts.reference_date
        AND stocks2018to2022.permno = valuesorts.permno
        '''
stocks2018to2022 = pd.read_sql(query, conn)
stocks2018to2022.drop('index', inplace = True, axis = 1)

print(stocks2018to2022)
print(stocks2018to2022.columns)

        permno                 date exchange       ret    market_cap  \
0        10026  2019-06-28 00:00:00   NASDAQ     0.373  3.030689e+03   
1        10026  2019-07-31 00:00:00   NASDAQ   15.4644  3.501411e+03   
2        10026  2019-08-30 00:00:00   NASDAQ    3.8851  3.637443e+03   
3        10026  2019-09-30 00:00:00   NASDAQ   -0.2901  3.627840e+03   
4        10026  2019-10-31 00:00:00   NASDAQ   -0.6458  3.604410e+03   
...        ...                  ...      ...       ...           ...   
115641   93436  2021-11-30 00:00:00   NASDAQ    2.7612  1.149642e+06   
115642   93436  2021-12-31 00:00:00   NASDAQ   -7.6855  1.092218e+06   
115643   93436  2022-01-31 00:00:00   NASDAQ  -11.3609  9.681319e+05   
115644   93436  2022-02-28 00:00:00   NASDAQ   -7.0768  8.996190e+05   
115645   93436  2022-03-31 00:00:00   NASDAQ      -100  1.113736e+06   

             reference_date     gvkey             datadate  book_value  \
0       2019-06-01 00:00:00   12825.0  2018-09-30 00:00:00   

In [ ]:
stocks2018to2022.to_csv('stocks2018to2022.csv')
!cp stocks2018to2022.csv "/content/gdrive/MyDrive/FinancePaperFIMA"

In [ ]:
stocks2018to2022['date'] = pd.to_datetime(stocks2018to2022['date'])
valueweightedret = stocks2018to2022.groupby(['date', 'size_portfolio', 'value_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights = pd.to_numeric(x['market_cap_weight'])))


In [ ]:
stocks2018to2022.to_sql("stocks2018to2022", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT date
        FROM stocks2018to2022
        '''
factors = pd.read_sql(query, conn)
factors['date'] = pd.to_datetime(factors['date'])

In [ ]:
factors['smb'] = ""
for i in factors.index:
  dateofdata = factors['date'][i]
  factors['smb'][i] = np.average(valueweightedret[dateofdata]["S"]) - np.average(valueweightedret[dateofdata]["B"])

valueweightedret = stocks2018to2022.groupby(['date', 'value_portfolio', 'size_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights = pd.to_numeric(x['market_cap_weight'])))

factors['hml'] = ""
for i in factors.index:
  dateofdata = factors['date'][i]
  factors['hml'][i] = np.average(valueweightedret[dateofdata]["H"]) - np.average(valueweightedret[dateofdata]["L"])

factors['mkt'] = ""
weightedavgmonthlyret = stocks2018to2022.groupby('date').apply(lambda x: np.average(pd.to_numeric(x.ret), weights = pd.to_numeric(x.market_cap_weight)))
for i in factors.index:
  dateofdata = factors['date'][i]
  factors['mkt'][i] = weightedavgmonthlyret[dateofdata]

<ipython-input-36-74bc66f800e7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  factors['smb'][i] = np.average(valueweightedret[dateofdata]["S"]) - np.average(valueweightedret[dateofdata]["B"])
<ipython-input-36-74bc66f800e7>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  factors['hml'][i] = np.average(valueweightedret[dateofdata]["H"]) - np.average(valueweightedret[dateofdata]["L"])
<ipython-input-36-74bc66f800e7>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

In [ ]:
factors['date'] = pd.to_datetime(factors['date'])
factors = factors[factors['date'] < "2022-03-31"]

factors.to_sql("factors", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT date, mkt, smb, hml
        FROM factors
        '''
factors = pd.read_sql(query, conn)

In [ ]:
factors['date'] = pd.to_datetime(factors['date'])
factors = factors.sort_values(by = ['date'], ascending = True)

factors = factors.reset_index()
factors.drop('index', inplace = True, axis = 1)

print(factors)
print(factors.columns)

         date        mkt       smb        hml
0  2019-06-28   7.442855  0.725381   0.434335
1  2019-07-31   1.451108 -2.028603   0.066145
2  2019-08-30  -2.374261 -2.802037  -4.746700
3  2019-09-30   1.867534  0.045370   5.694920
4  2019-10-31   2.251724 -0.015698  -2.206578
5  2019-11-29   4.040230  0.988982  -2.415970
6  2019-12-31   2.898891  1.488209   2.249337
7  2020-01-31  -0.200422 -3.625021  -6.096775
8  2020-02-28  -8.209507 -0.109146  -3.377045
9  2020-03-31 -14.038569 -7.288279 -12.349423
10 2020-04-30  14.391044  7.311321   2.131972
11 2020-05-29   5.059311  2.698112  -5.193029
12 2020-06-30   3.238252  4.809586  -2.446018
13 2020-07-31   5.810872 -1.980116  -1.637731
14 2020-08-31   7.379208  0.021343  -2.144586
15 2020-09-30  -3.638041 -0.115687  -2.046305
16 2020-10-30  -1.998925  3.976551   3.936080
17 2020-11-30  12.439364  7.063797   2.333686
18 2020-12-31   4.452327  5.474246  -2.392397
19 2021-01-29  -0.258749  6.875618   1.461469
20 2021-02-26   3.000820  4.254631

In [ ]:
file = filePath + "/FamaFrenchThreeFactors.csv"
factorsff = pd.read_csv(file)
print(factorsff)
print(factorsff.columns)

                                   Date  MktRF   SMB   HML
0                              19260701   0.10 -0.25 -0.27
1                              19260702   0.45 -0.33 -0.06
2                              19260706   0.17  0.30 -0.39
3                              19260707   0.09 -0.58  0.02
4                              19260708   0.21 -0.38  0.19
...                                 ...    ...   ...   ...
25316                               NaN    NaN   NaN   NaN
25317                               NaN    NaN   NaN   NaN
25318                               NaN    NaN   NaN   NaN
25319                               NaN    NaN   NaN   NaN
25320  Copyright 2022 Kenneth R. French    NaN   NaN   NaN

[25321 rows x 4 columns]
Index(['Date', 'MktRF', 'SMB', 'HML'], dtype='object')


In [ ]:
factorsff.to_sql("factorsff", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT Date as date, MktRF as mkt_ff, SMB as smb_ff, HML as hml_ff
        FROM factorsff
        '''
factorsff = pd.read_sql(query, conn)

In [ ]:
factorsff['mkt_ff'] = pd.to_numeric(factorsff['mkt_ff'], errors = 'coerce')
factorsff['smb_ff'] = pd.to_numeric(factorsff['smb_ff'], errors = 'coerce')
factorsff['hml_ff'] = pd.to_numeric(factorsff['hml_ff'], errors = 'coerce')

In [ ]:
factorsff = factorsff.dropna()

In [ ]:
print(factorsff)

           date  mkt_ff  smb_ff  hml_ff
0      19260701    0.10   -0.25   -0.27
1      19260702    0.45   -0.33   -0.06
2      19260706    0.17    0.30   -0.39
3      19260707    0.09   -0.58    0.02
4      19260708    0.21   -0.38    0.19
...         ...     ...     ...     ...
25310  20220825    1.45    0.18   -0.01
25311  20220826   -3.38   -0.55    1.69
25312  20220829   -0.72   -0.56    0.42
25313  20220830   -1.11   -0.25   -0.24
25314  20220831   -0.74    0.28   -0.44

[25315 rows x 4 columns]


In [ ]:
factors['date'] = pd.to_datetime(factors['date'])
factors['month'] = factors['date'].dt.month
factorsff['month'] = ""
for i in factorsff.index:
  factorsff['month'][i] = int(str(factorsff['date'][i])[-2] + str(factorsff['date'][i])[-1])

factors['year'] = factors['date'].dt.year
factorsff['year'] = ""
for i in factorsff.index:
  factorsff['year'][i] = int(str(factorsff['date'][i])[0:4])

<ipython-input-44-0d7f21bc567b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  factorsff['month'] = ""
<ipython-input-44-0d7f21bc567b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  factorsff['month'][i] = int(str(factorsff['date'][i])[-2] + str(factorsff['date'][i])[-1])
<ipython-input-44-0d7f21bc567b>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  factorsff['year'][i] = 

In [ ]:
factors.to_sql("factors", conn, if_exists = 'replace')
factorsff.to_sql("factorsff", conn, if_exists = 'replace')
query = '''
        SELECT factors.date, factors.mkt, factors.smb, factors.hml,
        factorsff.mkt_ff, factorsff.smb_ff, factorsff.hml_ff
        FROM factors
        LEFT JOIN factorsff
        ON factors.month = factorsff.month
        AND factors.year = factorsff.year
        '''
factors = pd.read_sql(query, conn)
print(factors)

                    date       mkt       smb       hml  mkt_ff  smb_ff  hml_ff
0    2019-06-28 00:00:00  7.442855  0.725381  0.434335   -0.84   -1.27    0.05
1    2019-06-28 00:00:00  7.442855  0.725381  0.434335   -0.39    0.60   -0.42
2    2019-06-28 00:00:00  7.442855  0.725381  0.434335   -0.22   -0.02    0.07
3    2019-06-28 00:00:00  7.442855  0.725381  0.434335   -0.05   -0.72    0.23
4    2019-06-28 00:00:00  7.442855  0.725381  0.434335   -0.02   -0.39    0.04
..                   ...       ...       ...       ...     ...     ...     ...
264  2022-02-28 00:00:00 -2.489926  1.966713  3.766222   -0.49    0.91   -1.40
265  2022-02-28 00:00:00 -2.489926  1.966713  3.766222    0.48   -1.74    0.72
266  2022-02-28 00:00:00 -2.489926  1.966713  3.766222    0.80    0.59   -1.15
267  2022-02-28 00:00:00 -2.489926  1.966713  3.766222    1.81    0.41    1.34
268  2022-02-28 00:00:00 -2.489926  1.966713  3.766222    2.09    0.84   -2.07

[269 rows x 7 columns]


In [ ]:
factors.describe(percentiles = [0.1, 0.25, 0.5, 0.75, 0.9])

,mkt,smb,hml,mkt_ff,smb_ff,hml_ff
count,269.000000,269.000000,269.000000,269.000000,269.000000,269.000000
mean,1.711211,0.257305,0.064149,0.178959,0.030335,0.049257
std,5.474745,3.582288,4.614579,1.526193,0.736497,1.353385
min,-14.038569,-7.288279,-12.349423,-9.630000,-2.490000,-5.000000
10%,-4.310052,-3.625021,-5.193029,-1.098000,-0.842000,-1.372000
25%,-1.767043,-2.373196,-2.392397,-0.300000,-0.440000,-0.640000
50%,2.826594,0.021343,-0.061248,0.240000,-0.010000,-0.040000
75%,4.452327,1.966713,3.679564,0.880000,0.500000,0.640000
90%,7.379208,5.474246,5.705383,1.514000,1.004000,1.700000
max,14.391044,7.311321,12.955060,7.060000,2.140000,6.740000


In [ ]:
ks_2samp(factors['mkt'], factors['mkt_ff'])
ks_2samp(factors['smb'], factors['smb_ff'])
ks_2samp(factors['hml'], factors['hml_ff'])

KstestResult(statistic=0.3308550185873606, pvalue=1.9666343420065377e-13, statistic_location=-2.046305019772791, statistic_sign=1)

In [ ]:
factors.corr(method = 'pearson')

,mkt,smb,hml,mkt_ff,smb_ff,hml_ff
mkt,1.000000,0.483468,0.217983,-0.017955,-0.117872,-0.014136
smb,0.483468,1.000000,0.206480,0.069644,-0.018831,-0.089178
hml,0.217983,0.206480,1.000000,0.025030,-0.050012,0.021129
mkt_ff,-0.017955,0.069644,0.025030,1.000000,0.192403,0.108726
smb_ff,-0.117872,-0.018831,-0.050012,0.192403,1.000000,0.173833
hml_ff,-0.014136,-0.089178,0.021129,0.108726,0.173833,1.000000


In [ ]:
factors.to_csv('FF3.csv')
!cp FF3.csv "/content/gdrive/MyDrive/FinancePaperFIMA"